In [20]:
import requests
import pandas as pd
from pytrends.request import TrendReq
import gradio as gr
import joblib
import datetime

# Load trained model
model = joblib.load("random_forest_model.pkl")  # or use "xgboost_model.joblib"

# Weather API info
WEATHER_API_KEY = "b607e1bfddef4d9f8e4192237251607"
WEATHER_API_URL = f"http://api.weatherapi.com/v1/current.json?q=Dhaka&key={WEATHER_API_KEY}"


In [21]:
def get_weather_data():
    response = requests.get(WEATHER_API_URL)
    data = response.json()
    
    temp = data['current']['temp_c']
    humidity = data['current']['humidity']
    rainfall = data['current'].get('precip_mm', 0.0)
    
    return temp, humidity, rainfall


In [22]:
def get_trend_scores():
    pytrends = TrendReq(hl='en-US', tz=360)
    keywords = ["onion", "পেঁয়াজ"]
    
    pytrends.build_payload(keywords, cat=0, timeframe='now 7-d', geo='BD')
    trend_df = pytrends.interest_over_time()
    
    if not trend_df.empty:
        trend_score_onion = trend_df["onion"].iloc[-1]
        trend_score_peyaj = trend_df["পেঁয়াজ"].iloc[-1]
    else:
        trend_score_onion = 50  # fallback values
        trend_score_peyaj = 50
    
    return trend_score_onion, trend_score_peyaj


In [23]:
def build_features():
    temp, humidity, rainfall = get_weather_data()
    trend_score_onion, trend_score_peyaj = get_trend_scores()

    rain_flag = 1 if rainfall > 0.5 else 0
    humidity_diff = 0
    temp_lag = temp
    rain_3day_sum = rainfall * 3  # crude estimate

    features = {
        "temp": temp,
        "humidity": humidity,
        "rainfall": rainfall,
        "rain_3day_sum": rain_3day_sum,
        "rain_flag": rain_flag,
        "humidity_diff": humidity_diff,
        "temp_lag": temp_lag,
        "trend_score_onion": trend_score_onion,
        "trend_score_পেঁয়াজ": trend_score_peyaj,
        "trend_score_পেঁয়াজ_lag1": trend_score_peyaj,
        "trend_score_পেঁয়াজ_avg3": trend_score_peyaj,
        "trend_score_পেঁয়াজ_spike": 0,
        "trend_score_onion_lag1": trend_score_onion,
        "trend_score_onion_avg3": trend_score_onion,
        "trend_score_onion_spike": 0
    }

    # Convert to DataFrame
    df = pd.DataFrame([features])

    # Ensure column order matches training
    feature_columns = joblib.load("model_features.joblib")
    df = df[feature_columns]

    return df


In [24]:
def predict_price():
    try:
        input_df = build_features()
        prediction = model.predict(input_df)[0]
        return f"📈 Predicted Onion Avg Price: {prediction:.2f} BDT"
    except Exception as e:
        return f"❌ Error: {str(e)}"


In [25]:
gr.Interface(
    fn=predict_price,
    inputs=[],
    outputs="text",
    title="📊 Onion Price Predictor (Dhaka)",
    description="Fetches live weather and Google Trends data to forecast the average price of onions in Dhaka."
).launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


C:\Users\bright_\anaconda3\Lib\site-packages\pytrends\request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
